## Задание 1.



In [1]:
import nltk
from gensim.models.phrases import Phraser, Phrases
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from string import punctuation

import numpy as np

start = '<start>'
end = '<end>'


def normalize(text):
    normalized_text = [word.strip(punctuation) for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i + n]))
    return ngrams


def get_dictionaries(unigrams, bigrams):
    id2word_unigrams = list(unigrams)
    word2id_unigrams = {word: i for i, word in enumerate(id2word_unigrams)}
    id2word_bigrams = list(bigrams)
    word2id_bigrams = {coll: d for d, coll in enumerate(id2word_bigrams)}
    return id2word_unigrams, word2id_unigrams, id2word_bigrams, word2id_bigrams


def generate(matrix, id2word_unigrams, word2id_bigrams, n=100, st=None):
    if st is None:
        st = [start, start]
    text = []
    current_bigram = st
    current_idx = word2id_bigrams[' '.join(current_bigram)]

    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word_unigrams[chosen])
        if id2word_unigrams[chosen] == end:
            current_bigram = st
        else:
            current_bigram = [current_bigram[1], id2word_unigrams[chosen]]

        current_idx = word2id_bigrams[' '.join(current_bigram)]

    return ' '.join(text)


dostoevsky = open('besy_dostoevsky.txt', encoding='cp1251').read()
tolstoy = open('anna_karenina.txt').read()
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

sentences_dostoevsky = [[start] + [start] + normalize(text) + [end] for text in sent_tokenize(dostoevsky)][:1000]
sentences_tolstoy = [[start] + [start] + normalize(text) + [end] for text in sent_tokenize(tolstoy)][:1000]


def print_generated_text(sentences):
    unigrams = Counter()
    bigrams = Counter()
    threegrams = Counter()

    for sentence in sentences:
        unigrams.update(sentence)
        bigrams.update(ngrammer(sentence))
        threegrams.update(ngrammer(sentence, n=3))

    len_unigrams = len(unigrams)
    matrix = np.zeros((
        len_unigrams,
        len_unigrams
    ))

    id2word_unigrams, word2id_unigrams, id2word_bigrams, word2id_bigrams \
        = get_dictionaries(unigrams, bigrams)

    for ngram in bigrams:
        word1, word2 = ngram.split()
        matrix[word2id_unigrams[word1]][word2id_unigrams[word2]] \
            = (bigrams[ngram] / unigrams[word1])

    len_bigrams = len(bigrams)
    matrix_final = np.zeros((
        len_bigrams,
        len_unigrams))

    for ngram in threegrams:
        word1, word2, word3 = ngram.split()
        bigram = f'{word1} {word2}'
        matrix_final[word2id_bigrams[bigram]][word2id_unigrams[word3]] \
            = (threegrams[ngram] / bigrams[bigram])

    print(generate(matrix_final, id2word_unigrams, word2id_bigrams).replace(end, '\n'))


print("# generated text for dostoevsky")
print_generated_text(sentences_dostoevsky)
print("# generated text for tolstoy")
print_generated_text(sentences_tolstoy)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# generated text for dostoevsky
замечу от себя что действительно у многих особ в генеральских чинах есть привычка смешно говорить «я служил государю моему…» то есть просто в хандру но словечко это нравилось многоуважаемой варваре петровне довольно странные слухи молодой человек готовился поступить в университет 
 разъезжая по визитам nicolas был арестован и отведен покамест на гауптвахту где и заперт в особую каморку с особым часовым у дверей 
 — к после матери не имевшей детей во втором супружестве 
 вдруг говорит мне что все время разлуки с своим другом провел в крайнем унынии 
 сцена открывается хором женщин потом хором мужчин потом каких-то сил
# generated text for tolstoy
левин тоже не мог понять к чему все эти предметы каких держалось большинство 
 – проговорил степан аркадьич который любил физиологию совершенно невольно вдруг улыбнулось привычною доброю и потому вышел из комнаты 
 я никогда ни с кем не говорил об этом 
 николай левин» 
 всю дорогу приятели молчали 
 – не в силах

Тексты, сгенерированные такой программой, выглядят более осмысленными, чем тексты, сгенерированные на основе биграммной модели.
Например, у меня получилось предложение: "замечу от себя что действительно у многих особ в генеральских чинах есть привычка смешно говорить «я служил государю моему…» то есть просто в хандру но словечко это нравилось многоуважаемой варваре петровне". Конечно, его трудно назвать 
полностью законченным и осмысленным высказыванием, но оно явно лучше выглядит, если сравнить его с любым примером из семинарской тетрадки: "пошли как вы считаете поступок в голове» но он подсюсюкивает и те минуты еще не посмотрел николай всеволодович громко истину ".
 


## Задание 2

In [5]:
def get_sentences(text=''):
    return sent_tokenize(text.replace('\n', ' '), 'russian')[:10]


def ngrammer(tokens, stops=None, n=5):
    if stops is None:
        stops = set(stopwords.words('russian') + ['–'])
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0, len(tokens) - n + 1):
        ngrams.append('_'.join(tokens[i:i + n]))
    return ngrams


text = open('anna_karenina.txt').read()[:10000]
# 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на отдельные символы (не
# потеряйте пробелы)
sentences = get_sentences(text)
tokens = [list(s) for s in sentences]

# 2) обучите gensim.models.Phrases на полученных символьных предложениях
phrases1 = Phrases(tokens, scoring='npmi', threshold=-1, min_count=1)
phrases2 = Phrases(phrases1[tokens], scoring='npmi', threshold=-1)
phrases3 = Phrases(phrases2[phrases1[tokens]], scoring='npmi', threshold=-1)
phrases4 = Phrases(phrases3[phrases2[phrases1[tokens]]], scoring='npmi', threshold=-1)

# 3) примените полученный нграммер к этим символьным предложениям
_tokens = list(phrases4[phrases3[phrases2[phrases1[tokens]]]])
print(_tokens)
ngrams = [ngrammer(t) for t in _tokens]
print(ngrams)

[['A_n', 'n_o', 't_a', 't_i', 'o_n', ' _ ', ' _«', 'А_н', 'н_а', ' _К', 'а_р', 'е_н', 'и_н', 'а_»', ' _–', ' _э', 'т_о', ' _с', 'л_о', 'ж_н', 'о_е', ',_ ', 'п_с', 'и_х', 'о_л', 'о_г', 'и_ч', 'е_с', 'к_и', ' _у', 'т_о', 'н_ч', 'е_н', 'н_о', 'е_,', ' _о', 'с_т', 'р_о', 'п_р', 'о_б', 'л_е', 'м_н', 'о_е', ' _п', 'р_о', 'и_з', 'в_е', 'д_е', 'н_и', 'е_,', ' _н', 'а_с', 'ы_щ', 'е_н', 'н_о', 'е_ ', 'п_р', 'и_м', 'е_т', 'а_м', 'и_ ', 'в_р', 'е_м', 'е_н', 'и_.'], ['Л_.', 'Н_.', ' _Т', 'о_л', 'с_т', 'о_й', ' _н', 'а_ ', 'с_т', 'р_а', 'н_и', 'ц_а', 'х_ ', 'п_р', 'о_и', 'з_в', 'е_д', 'е_н', 'и_я', ' _п', 'о_к', 'а_з', 'ы_в', 'а_е', 'т_,', ' _к', 'а_к', ' _р', 'у_ш', 'а_т', 'с_я', ' _о', 'с_т', 'а_т', 'к_и', ' _п', 'а_т', 'р_и', 'а_р', 'х_а', 'л_ь', 'н_о', 'г_о', ' _у', 'к_л', 'а_д', 'а_ ', 'ж_и', 'з_н', 'и_ ', 'в_ ', 'Р_о', 'с_с', 'и_и', ' _п', 'о_д', ' _н', 'а_т', 'и_с', 'к_о', 'м_ ', 'б_у', 'р_ж', 'у_а', 'з_н', 'о_г', 'о_ ', 'п_р', 'о_г', 'р_е', 'с_с', 'а_,', ' _к', 'а_к', ' _п', 'а_д', 'а_ю', 'т